<a href="https://colab.research.google.com/github/junveren/autonews/blob/main/autonews_6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install feedparser vaderSentiment yfinance ics


In [ ]:
from ics import Calendar, Event

In [ ]:
# Colab Script — Dashboard with API Ninjas Earnings Dates

import os
import csv
import feedparser
import urllib.parse
from datetime import datetime, timedelta
import pandas as pd
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from google.colab import drive
from getpass import getpass
import base64
import requests
import yfinance as yf



# === USER CONFIG ===
GITHUB_USERNAME = "junveren"
REPO_NAME = "autonews"
BRANCH = "main"
TARGET_FILE = "weekly_sentiment_log.html"
GITHUB_TOKEN = getpass("Paste your GitHub token (hidden):")
API_NINJAS_KEY = getpass("Paste your API Ninjas key (hidden):")

# === CONFIG ===
COMPANY_QUERIES = {
    "Ford": [("Ford stock", "stock"), ("Ford Motor Company", "connected car")],
    "Toyota": [("Toyota stock", "stock"), ("Toyota", "tech")],
    "GM": [("General Motors stock", "stock"), ("General Motors", "tech")],
    "Tesla": [("Tesla stock", "stock"), ("Tesla", "tech")],
    "Harley-Davidson": [("Harley-Davidson stock", "stock"), ("Harley-Davidson", "tech")],
    "Polaris": [("Polaris Inc stock", "stock"), ("Polaris", "tech")],
    "Visteon": [("Visteon stock", "stock"), ("Visteon", "tech")],
    "Rivian": [("Rivian stock", "stock"), ("Rivian", "tech")],
    "Nvidia": [("Nvidia stock", "stock"), ("Nvidia", "tech")],
    "Qualcomm": [("Qualcomm stock", "stock"), ("Qualcomm", "tech")],
    "Honda": [("Honda stock", "stock"), ("Honda", "tech")],
    "Lucid": [("Lucid Motors stock", "stock"), ("Lucid", "tech")],
    "Stellantis": [("Stellantis stock", "stock"), ("Stellantis", "tech")],
    "Subaru": [("Subaru stock", "stock"), ("Subaru", "tech")],
    "Panasonic": [("Panasonic stock", "stock"), ("Panasonic Automotive", "automotive")],
    "Bosch": [("Bosch stock", "stock"), ("Bosch Automotive", "automotive")],
    "Nissan": [("Nissan stock", "stock"), ("Nissan", "tech")],
    "Harman": [("Harman stock", "stock"), ("Harman", "tech")],
    "Scout": [("Scout stock", "stock"), ("ScoutMotors", "tech")],
    "Mitsubishi": [("Mitsubishi stock", "stock"), ("Mitsubishi", "tech")],
    "HyundaiKia": [("HyundaiKia stock", "stock"), ("HyundaiKia", "tech")],
    "Telenav": [("Telenav stock", "stock"), ("Telenav", "tech")],
}

TICKER_MAP = {
    "Ford": "F", "Toyota": "TM", "GM": "GM", "Tesla": "TSLA",
    "Harley-Davidson": "HOG", "Polaris": "PII", "Visteon": "VC", "Rivian": "RIVN",
    "Nvidia": "NVDA", "Qualcomm": "QCOM", "Honda": "HMC", "Lucid": "LCID",
    "Stellantis": "STLA", "Subaru": "FUJHY", "Panasonic": "PCRFY", "Bosch": None,
    "Nissan": "NSANY", "Harman": None, "Scout": None, "Mitsubishi": "MSBHF",
    "HyundaiKia": "HYMTF", "Telenav": None
}

KEYWORDS = list(set([
    "SYNC","SYNC4","ADAS","navigation","connected car","infotainment","ADASIS","BlueCruise","Starlink","hp map","sd map","autostream","multimedia",
    "SmartCockpit","digital cockpit","Uconnect","Snapdragon Ride","SmartCore","DreamDrive","Driver+","recall","FNV4","FNV3","SAE","GDPR",
    "Sensing","SkipGen","hazards","ISA","Copilot","CarPlay","AndroidAuto","Mapbox","orbis","Hyperion","google automotive", "car play",
    "edge computing","software defined vehicle","cockpit domain","R1H","R2","FSD","Autopilot","HERE","probedata","traffic","map","ev charger",
    "Teammate","real-time traffic","charge point","Snapdragon","Ride Command","Boom","TomTom","Software","software","machine learning","connectivity",
]))

# === INIT ===
drive.mount('/content/drive')
os.makedirs("/content/drive/MyDrive/AutoNews", exist_ok=True)
today_str = datetime.now().strftime('%Y-%m-%d')
HTML_FILE = f"/content/drive/MyDrive/AutoNews/{TARGET_FILE}"
analyzer = SentimentIntensityAnalyzer()

# === API Ninjas Earnings ===
def get_earnings_date_api_ninjas(ticker):
    try:
        url = f"https://api.api-ninjas.com/v1/earningscalendar?ticker={ticker}&show_upcoming=true"
        headers = {"X-Api-Key": API_NINJAS_KEY}
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            data = res.json()
            future = [d for d in data if 'date' in d and d['date'] >= today_str]
            if future:
                nearest = sorted(future, key=lambda d: d['date'])[0]
                return nearest['date']
        return "N/A"
    except:
        return "N/A"

# === Revenue Growth (Yahoo Finance) ===
def get_revenue_growth(ticker):
    try:
        ticker_obj = yf.Ticker(ticker)
        fin = ticker_obj.quarterly_income_stmt
        print(f"\n=== {ticker} ===")
        print("Labels:", list(fin.index))
        print("Total Revenue:", fin.loc["Total Revenue"].to_dict())
        if "Total Revenue" not in fin.index or fin.loc["Total Revenue"].isna().sum() > len(fin.columns) - 2:
            return "N/A"
        values = fin.loc["Total Revenue"].dropna()
        if len(values) < 2:
            return "N/A"
        current, previous = values.iloc[0], values.iloc[1]
        change = ((current - previous) / previous) * 100
        color = "green" if change > 0 else "red"
        return f"<span style='color:{color}'>{abs(change):.1f}%</span>"
    except Exception as e:
        print(f"{ticker} Error:", e)
        return "N/A"



 #✅ Render Sentiment Table

def render_company_table(oem_avg):
    rows = ""
    for _, row in oem_avg.iterrows():
        company = row['company']
        sentiment = row['sentiment']
        ticker = TICKER_MAP.get(company)
        earnings = get_earnings_date_api_ninjas(ticker) if ticker else "N/A"
        revenue = get_revenue_growth(ticker) if ticker else "N/A"
        rows += f"<tr><td>{company}</td><td style='text-align:center'>{sentiment}</td><td style='text-align:center'>{revenue}</td><td style='text-align:center'>{earnings}</td></tr>"
    return f"""
    <table class='table-sm sentiment-table' border='0'>
    <thead><tr><th>Company</th><th>Avg Weekly Sentiment</th><th>Quarterly Revenue Growth</th><th>Next Earnings Date</th></tr></thead>
    <tbody>{rows}</tbody>
    </table>
    """


def fetch_news_rss(query):
    encoded = urllib.parse.quote(query)
    return feedparser.parse(f"https://news.google.com/rss/search?q={encoded}").entries[:10]

def match_keywords(text):
    matched = [kw for kw in KEYWORDS if re.search(r'\b' + re.escape(kw.lower()) + r'\b', text.lower())]
    return len(matched), matched

def get_sentiment(text):
    return round(analyzer.polarity_scores(text)['compound'], 3)

def article_block(row):
    return f"<li class='article-item'><b>{row['company']}</b>: {row['title']}<br><a href='{row['link']}' target='_blank'>Full article</a> <span class='sentiment'>({row['sentiment']})</span></li>"

# 🔁 Insert this block before `main()` to fetch from GNews.io
from getpass import getpass

GNEWS_API_KEY = getpass("Paste your GNews API key (hidden):")

def fetch_news_gnews(query):
    url = f"https://gnews.io/api/v4/search?q={urllib.parse.quote(query)}&lang=en&max=10&token={GNEWS_API_KEY}"
    try:
        r = requests.get(url)
        if r.status_code == 200:
            return r.json().get("articles", [])
    except:
        pass
    return []

# 🔁 Update `main()` loop to combine RSS + GNews

def main():
    global df
    all_news = []
    seen_titles, seen_links = set(), set()
    cutoff = datetime.utcnow() - timedelta(days=5)

    for company, queries in COMPANY_QUERIES.items():
        for query, qtype in queries:
            # RSS FEED
            for entry in fetch_news_rss(query):
                title = entry.get('title', '')
                link = entry.get('link', '')
                summary = entry.get('summary') or entry.get('description') or ''
                pub_time = entry.get('published_parsed')
                if not title or not link or not pub_time:
                    continue
                if title in seen_titles or link in seen_links:
                    continue
                pub_dt = datetime(*pub_time[:6])
                if pub_dt < cutoff:
                    continue

                full_text = f"{title} {summary}"
                priority_score, matched_keywords = match_keywords(full_text)
                sentiment_score = get_sentiment(full_text)

                all_news.append([
                    datetime.now().strftime("%Y-%m-%d %H:%M"),
                    company, title, link, summary,
                    sentiment_score, priority_score, "; ".join(matched_keywords), qtype, "RSS"
                ])
                seen_titles.add(title)
                seen_links.add(link)

            # GNEWS API
            for article in fetch_news_gnews(query):
                title = article.get("title", '')
                link = article.get("url", '')
                summary = article.get("description", '')
                if not title or not link:
                    continue
                if title in seen_titles or link in seen_links:
                    continue
                pub_dt = article.get("publishedAt")
                if pub_dt:
                    pub_dt = datetime.strptime(pub_dt[:19], "%Y-%m-%dT%H:%M:%S")
                    if pub_dt < cutoff:
                        continue

                full_text = f"{title} {summary}"
                priority_score, matched_keywords = match_keywords(full_text)
                sentiment_score = get_sentiment(full_text)

                all_news.append([
                    datetime.now().strftime("%Y-%m-%d %H:%M"),
                    company, title, link, summary,
                    sentiment_score, priority_score, "; ".join(matched_keywords), qtype, "GNews"
                ])
                seen_titles.add(title)
                seen_links.add(link)

    df = pd.DataFrame(all_news, columns=[
        "datetime", "company", "title", "link", "summary",
        "sentiment", "priority_score", "matched_keywords", "type", "source"
    ])

    print("\n✅ Article sources:")
    print(df["source"].value_counts())

    #


    df_display = df.drop(columns=["link", "type"])
    full_table = df_display.to_html(index=False, escape=False, justify="center", classes="table table-striped", border=0)

    avg_sentiment = round(df["sentiment"].mean(), 3)
    sentiment_icon = "👍" if avg_sentiment >= 0.2 else "👎" if avg_sentiment <= -0.2 else "😐"

    oem_avg = df.groupby("company")["sentiment"].mean().round(3).reset_index()
    oem_avg = oem_avg.sort_values("sentiment", ascending=False)
    oem_html = render_company_table(oem_avg)


    top_tech = df[df["type"] == "tech"].sort_values("sentiment", ascending=False).head(2)
    top_stock = df[df["type"] == "stock"].sort_values("sentiment", ascending=False).head(1)
    top3 = pd.concat([top_tech, top_stock])

    bottom_tech = df[df["type"] == "tech"].sort_values("sentiment").head(2)
    bottom_stock = df[df["type"] == "stock"].sort_values("sentiment").head(1)
    bottom3 = pd.concat([bottom_tech, bottom_stock])

    top_html = "".join([article_block(r) for _, r in top3.iterrows()])
    bottom_html = "".join([article_block(r) for _, r in bottom3.iterrows()])


    ...
    html_content = f"""
<!DOCTYPE html>
<html lang='en'>
<head>
<!-- Google tag (gtag.js) -->
<script async src="https://www.googletagmanager.com/gtag/js?id=G-8N2ETJT89L"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){{dataLayer.push(arguments);}}
  gtag('js', new Date());
  gtag('config', 'G-8N2ETJT89L');
</script>

    <meta charset='UTF-8'>
    <title>America's Auto News – {today_str}</title>
    <style>
    body {{
        font-family: Helvetica, Arial;
        background: #ffffff;
        color: #121212;
        margin: 30px 40px;
        transition: background 0.3s ease, color 0.3s ease;
    }}

    a {{ color: #0077cc; }}
    table {{
        width: 100%;
        border-collapse: collapse;
        margin-top: 30px;
    }}

    th, td {{
        padding: 10px;
        border: 1px solid #ccc;
        text-align: center;
    }}

    th {{
        background-color: #e3e3e3;
        color: #000;
    }}

    .table-striped tr:nth-child(even) {{
        background-color: #f9f9f9;
    }}

    .dark-mode {{
        background: #121212;
        color: #ffffff;
    }}
    .dark-mode a {{ color: #6EC6FF; }}
    .dark-mode th {{ background-color: #37518e; color: #fff; }}
    .dark-mode td, .dark-mode th {{ border: 1px solid #444; }}
    .dark-mode .table-striped tr:nth-child(even) {{ background-color: #2a2a2a; }}

    .theme-toggle-wrapper {{
        position: absolute;
        top: 10px;
        right: 10px;
        display: flex;
        align-items: center;
    }}
    .theme-toggle {{
        width: 60px;
        height: 30px;
        background: #000000;
        border-radius: 30px;
        position: relative;
        cursor: pointer;
        box-shadow: 0 0 10px rgba(0,0,0,0.15);
        transition: background 0.3s ease;
        margin-left: 8px;
    }}
    .knob {{
        width: 30px;
        height: 30px;
        background: #f4a261;
        border-radius: 50%;
        position: absolute;
        top: 0;
        left: 0;
        transition: all 0.3s ease;
        display: flex;
        align-items: center;
        justify-content: center;
    }}
    .icon {{ font-size: 16px; transition: opacity 0.3s ease; }}
    .moon {{ opacity: 0; }}
    .dark-mode .theme-toggle {{ background: #ffffff; }}
    .dark-mode .knob {{ left: 30px; background: #264653; }}
    .dark-mode .moon {{ opacity: 1; }}
    .feedback-buttons {{
        display: flex;
        justify-content: center;
        gap: 20px;
        margin-top: 80px;
    }}
    .feedback-buttons button {{
        font-size: 30px;
        padding: 10px 20px;
        cursor: pointer;
    }}
    .thank-you {{
        text-align: center;
        margin-top: 20px;
        font-size: 18px;
        display: none;
    }}
    </style>
</head>
<body>
    <div class="theme-toggle-wrapper">
        <span>Dark Mode</span>
        <div class="theme-toggle" onclick="toggleTheme()">
            <div class="knob">
                <div class="icon moon">🌙</div>
            </div>
        </div>
    </div>

    <h1>America's Auto News – {today_str}</h1>
    <p>Average Sentiment this week: <b>{avg_sentiment}</b> {sentiment_icon}</p>

    <div class='top-articles'>
        <h2>📈 Top 3 Positive Articles</h2>
        <div class='article-blocks'>{top_html}</div>
    </div>

    <div class='bottom-articles'>
        <h2>📉 Top 3 Negative Articles</h2>
        <div class='article-blocks'>{bottom_html}</div>
    </div>

    <h2>🏎️ Average Sentiment Per Company</h2>
    <p><i>Sentiment scores range from −1 to +1 using VADER. Sorted by most positive.</i></p>
    {oem_html}

    <h2>📰 Full Article Listing</h2>
    {full_table}

    <div class="feedback-buttons" id="feedback-section">
        <button onclick="submitFeedback('up')">👍</button>
        <button onclick="submitFeedback('down')">👎</button>
    </div>
    <div class="thank-you" id="thank-you-msg">Thank you for your feedback!</div>

    <script>
    function toggleTheme() {{
        const body = document.body;
        const knob = document.querySelector('.knob');
        const moon = document.querySelector('.moon');
        body.classList.toggle('dark-mode');
        knob.style.left = body.classList.contains('dark-mode') ? '30px' : '0px';
        moon.style.opacity = body.classList.contains('dark-mode') ? '1' : '0';
    }}
    function submitFeedback(choice) {{
        document.getElementById("feedback-section").style.display = "none";
        document.getElementById("thank-you-msg").style.display = "block";
    }}
    </script>
</body>
</html>
"""



    with open(HTML_FILE, "w", encoding="utf-8") as f:
        f.write(html_content)
    print("✅ HTML dashboard created!")
#  Save CSV version of the news data
    import os # Add this line at the beginning of the notebook or before this snippet
#  Save CSV version of the news data
    csv_path = "/content/drive/MyDrive/AutoNews/master_news_sentiment.csv"
    if os.path.exists(csv_path):
      df.to_csv(csv_path, mode='a', header=False, index=False)
    else:
      df.to_csv(csv_path, index=False)
    print(f"✅ CSV file saved to: {csv_path}")

    return top3, bottom3, df, avg_sentiment, sentiment_icon, oem_html, full_table, top_html, bottom_html

    with open(HTML_FILE, "rb") as f:
        content = base64.b64encode(f.read()).decode()

    api_url = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPO_NAME}/contents/{TARGET_FILE}"
    headers = {"Authorization": f"token {GITHUB_TOKEN}"}
    r = requests.get(api_url, headers=headers)

    payload = {
        "message": f"Update {TARGET_FILE} – {today_str}",
        "content": content,
        "branch": BRANCH,
    }

    if r.status_code == 200:
        payload["sha"] = r.json()["sha"]

    res = requests.put(api_url, headers=headers, json=payload)

    if res.status_code in [200, 201]:
        print(f"🚀 Pushed to GitHub: https://{GITHUB_USERNAME}.github.io/{REPO_NAME}/{TARGET_FILE}")
    else:
        print(f"❌ GitHub push failed: {res.status_code}")
        print(res.json())


#  COPY-PASTE HTML OUTPUT FOR OUTLOOK
top3, bottom3, df, avg_sentiment, sentiment_icon, oem_html, full_table, top_html, bottom_html = main()
# Build best and worst article per company
per_company = {}
for company in df["company"].unique():
    sub_df = df[df["company"] == company]
    if sub_df.empty:
        continue
    best_row = sub_df.loc[sub_df["sentiment"].idxmax()]
    worst_row = sub_df.loc[sub_df["sentiment"].idxmin()]
    per_company[company] = {
        "best_title": best_row["title"],
        "best_link": best_row["link"],
        "worst_title": worst_row["title"],
        "worst_link": worst_row["link"]
    }


html_lines = [
    f"<h3>America's Auto News – {today_str}</h3>",
    f'<p>📎 <a href="https://junveren.github.io/autonews/weekly_sentiment_log.html">Read Full News and More</a><br>',
    f'📘 <a href="https://tomtom.atlassian.net/wiki/spaces/AMERAUT/pages/629628168/Automotive+News+Dashboard">About This Dashboard</a></p>',
    f'💾 <a href="https://tomtominternational.sharepoint.com/:x:/r/sites/AmericasAutomotive/_layouts/15/Doc.aspx?sourcedoc=%7B8B935B0A-960A-4452-88D1-01B9A72DC002%7D&file=master_news_sentiment%20.xlsx&action=default&mobileredirect=true">News Archive</a></p>',
    "<h3>📈 Top 3 Positive Articles</h3><ul>"
]

for _, row in top3.iterrows():
    html_lines.append(f'<li><a href="{row["link"]}">{row["title"]}</a> ({row["company"]})</li>')
html_lines.append("</ul><h3>📉 Top 3 Negative Articles</h3><ul>")

for _, row in bottom3.iterrows():
    html_lines.append(f'<li><a href="{row["link"]}">{row["title"]}</a> ({row["company"]})</li>')
html_lines.append("</ul><h3>📰 Top Articles Per Company</h3>")

for company, val in per_company.items():
    html_lines.append(f"<p><b>{company}</b><br>")
    html_lines.append(f'• <a href="{val["best_link"]}">{val["best_title"]}</a><br>')
    html_lines.append(f'• <a href="{val["worst_link"]}">{val["worst_title"]}</a></p>')

# Join all and print
html_summary = "\n".join(html_lines)
print(html_summary)

# Save as .html file for Outlook copy-paste
html_path = "/content/drive/MyDrive/AutoNews/outlook_summary.html"
with open(html_path, "w", encoding="utf-8") as f:
    f.write(html_summary)

print(f"\n📧 Outlook-ready summary saved at:\n{html_path}")



